## Saving a Cats v Dogs Model in ONNX

Cloned from Fast.ai Course22, https://github.com/fastai/course22/blob/master/02-saving-a-basic-fastai-model.ipynb

Builds upon the Fast.ai Cats v Dogs notebook to save the model in ONNX.  The notebook shows how easy it is to save a Fast.ai model to ONNX.  Once the model is in ONNX, it can be converted to other machine learning frameworks like Elixir's Axon, https://github.com/elixir-nx/axon_onnx

In [1]:
# Make sure we've got the latest version of fastai:
# !pip install -Uqq fastai

First, import all the stuff we need from fastai:

In [2]:
from fastai.vision.all import *

Download and decompress our dataset, which is pictures of dogs and cats:

In [3]:
path = untar_data(URLs.PETS)/'images'

We need a way to label our images as dogs or cats. In this dataset, pictures of cats are given a filename that starts with a capital letter:

In [4]:
def is_cat(x): return x[0].isupper() 

Now we can create our `DataLoaders`:

In [5]:
dls = ImageDataLoaders.from_name_func('.',
    get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192))

... and train our model, a resnet18 (to keep it small and fast):

In [6]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10, cbs=EarlyStoppingCallback(patience=2))

/home/ml3/miniconda3/envs/fastai_22/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  warnings.warn(
/home/ml3/miniconda3/envs/fastai_22/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,error_rate,time
0,0.197886,0.060117,0.018945,00:13


epoch,train_loss,valid_loss,error_rate,time
0,0.068240,0.037473,0.012855,00:10
1,0.040520,0.031718,0.010149,00:10
2,0.037268,0.043240,0.012855,00:10
3,0.035163,0.063045,0.014885,00:10


No improvement since epoch 1: early stopping


Now we can export our trained `Learner`. This contains all the information needed to run the model:

In [7]:
learn.save('cats_v_dogs')

Path('models/cats_v_dogs.pth')

In [8]:
learn.load('cats_v_dogs')

In [9]:
model = learn.model
model.cuda()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [10]:
import torch
import torchvision

In [11]:
dummy_input = torch.randn(10, 3, 224, 224, device="cuda")

In [12]:
# From https://pytorch.org/docs/stable/onnx.html#example-alexnet-from-pytorch-to-onnx

# Providing input and output names sets the display names for values
# within the model's graph. Setting these does not change the semantics
# of the graph; it is only for readability.
#
# The inputs to the network consist of the flat list of inputs (i.e.
# the values you would pass to the forward() method) followed by the
# flat list of parameters. You can partially specify names, i.e. provide
# a list here shorter than the number of inputs to the model, and we will
# only set that subset of names, starting from the beginning.


input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "onnx/cats_v_dogs.onnx", verbose=True, input_names=input_names, output_names=output_names)

Exported graph: graph(%actual_input_1 : Float(10, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cuda:0),
      %1.2.weight : Float(1024, strides=[1], requires_grad=1, device=cuda:0),
      %1.2.bias : Float(1024, strides=[1], requires_grad=1, device=cuda:0),
      %1.2.running_mean : Float(1024, strides=[1], requires_grad=0, device=cuda:0),
      %1.2.running_var : Float(1024, strides=[1], requires_grad=0, device=cuda:0),
      %1.6.weight : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %1.6.bias : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %1.6.running_mean : Float(512, strides=[1], requires_grad=0, device=cuda:0),
      %1.6.running_var : Float(512, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_218 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_219 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_221 : Float(64, 64, 3, 3, strides=

You can now download the ONNX file and import it into a framework of your choice.